In [58]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_selector
import numpy as np



In [13]:
files = [
  'SB11_20191.xlsx',
  'SB11_20201.xlsx',
  'SB11_20211.xlsx',
  'SB11_20221.xlsx',
  'SB11_20231.xlsx'
]

In [61]:
icfes_data_years = []
for x in files:
  print(f"data/{x}")
  icfes_data_years.append(pd.read_excel(f"data/{x}"))

data/SB11_20191.xlsx
data/SB11_20201.xlsx
data/SB11_20211.xlsx
data/SB11_20221.xlsx
data/SB11_20231.xlsx


In [62]:
#Create base DF
SB_data = pd.concat(icfes_data_years)
SB_data

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,PERIODO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_PAIS_RESIDE,ESTU_TIENEETNIA,ESTU_DEPTO_RESIDE,...,ESTU_NSE_INDIVIDUAL,ESTU_NSE_ESTABLECIMIENTO,ESTU_ESTADOINVESTIGACION,ESTU_GENERACIONE,ESTU_GENERACION-E,PERCENTIL_ESPECIAL_GLOBAL,ESTU_AGREGADO,ESTU_PRESENTACIONSABADO,SEED_CODIGOMEN,SEED_NOMBRE
0,TI,COLOMBIA,F,2000-12-07 00:00:00,20191,SB11201910008548,ESTUDIANTE,COLOMBIA,No,VALLE,...,3.0,3.0,PUBLICAR,NO,NaN,NaN,NaN,NaN,NaN,NaN
1,CC,COLOMBIA,F,1998-03-23 00:00:00,20191,SB11201910004475,ESTUDIANTE,COLOMBIA,No,VALLE,...,2.0,3.0,PUBLICAR,NO,NaN,NaN,NaN,NaN,NaN,NaN
2,TI,COLOMBIA,M,2001-03-22 00:00:00,20191,SB11201910011427,ESTUDIANTE,COLOMBIA,No,VALLE,...,2.0,3.0,PUBLICAR,NO,NaN,NaN,NaN,NaN,NaN,NaN
3,CC,COLOMBIA,M,1994-06-16 00:00:00,20191,SB11201910041975,ESTUDIANTE,COLOMBIA,No,VALLE,...,3.0,3.0,PUBLICAR,NO,NaN,NaN,NaN,NaN,NaN,NaN
4,CC,COLOMBIA,F,2000-09-21 00:00:00,20191,SB11201910014490,ESTUDIANTE,COLOMBIA,No,BOGOTA,...,4.0,4.0,PUBLICAR,NO,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12699,TI,COLOMBIA,F,2005-10-29 00:00:00,20231,SB11202310000043,ESTUDIANTE,COLOMBIA,No,BOGOTÁ,...,NaN,NaN,PUBLICAR,NaN,NaN,NaN,NaN,No,68.0,BOGOTA
12700,TI,COLOMBIA,F,2005-08-15 00:00:00,20231,SB11202310001190,ESTUDIANTE,COLOMBIA,No,BOGOTÁ,...,NaN,NaN,PUBLICAR,NaN,NaN,NaN,NaN,No,68.0,BOGOTA
12701,CC,COLOMBIA,F,2004-06-25 00:00:00,20231,SB11202310000099,ESTUDIANTE,COLOMBIA,No,BOGOTÁ,...,NaN,NaN,PUBLICAR,NaN,NaN,NaN,NaN,No,68.0,BOGOTA
12702,CC,COLOMBIA,F,2004-11-01 00:00:00,20231,SB11202310001185,ESTUDIANTE,COLOMBIA,No,BOGOTÁ,...,NaN,NaN,PUBLICAR,NaN,NaN,NaN,NaN,No,68.0,BOGOTA


### Column Dropper Custom
- scikit-learn no proporciona un transformador integrado específicamente para eliminar columnas directamente por sus nombres. 
- Al incluir la eliminación de columnas como un paso dentro de un pipeline, se asegura que los pasos de preprocesamiento de datos se apliquen de manera consistente.
- Un eliminador de columnas personalizado asegura que las mismas columnas se eliminen cada vez que se ejecute el pipeline, manteniendo la consistencia y reproducibilidad del proceso de entrenamiento de modelos.

In [60]:
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop=None):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_transformed = X.drop(columns=self.columns_to_drop, errors='ignore')
        return X_transformed


### Data frame wrapper custom
Por defecto, muchas transformaciones en scikit-learn devuelven arrays de NumPy en lugar de DataFrames de pandas. Esto puede ser un inconveniente cuando se desea mantener los nombres de las columnas y el índice del DataFrame original, ya que estas características se pierden al convertirse a un array de NumPy. El DataFrameWrapper permite convertir el resultado de la transformación de vuelta a un DataFrame, preservando los nombres de las columnas y los índices.

In [59]:
# Data needs to be transformed before it can be used for training a model.
# this class is a convenient way to use a transformer while keeping the data in a pandas DataFrame 
# and preserving the original feature names and row labels.
class DataFrameWrapper(BaseEstimator, TransformerMixin):
    # Takes a transformer object as an argument.
    def __init__(self, transformer):
        self.transformer = transformer

    def fit(self, X, y=None):
        self.transformer.fit(X, y)
        self.columns_ = self.transformer.get_feature_names_out()
        return self

    def transform(self, X):
        X_transformed = self.transformer.transform(X)
        return pd.DataFrame(X_transformed, columns=self.columns_, index=X.index)


In [85]:

columns_to_drop = [
    'ESTU_INSE_INDIVIDUAL',
    'ESTU_NSE_INDIVIDUAL',
    'ESTU_NSE_ESTABLECIMIENTO',
    'ESTU_GENERACIONE',
    'ESTU_GENERACION-E',
    'PERCENTIL_ESPECIAL_GLOBAL',
    'ESTU_AGREGADO',
    'ESTU_PRESENTACIONSABADO',
    'SEED_CODIGOMEN',
    'SEED_NOMBRE'
]

numeric_data_imputer = DataFrameWrapper(ColumnTransformer(
    transformers=[
        ('num_imputer', SimpleImputer(strategy='median'), make_column_selector(dtype_include=np.number))
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
))

categorical_data_imputer = DataFrameWrapper(ColumnTransformer(
    transformers=[
        ('cat_imputer', SimpleImputer(strategy='most_frequent'), make_column_selector(dtype_include=object))
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
))

data_cleaning_pipeline = Pipeline(steps=[
    ('column_dropper', ColumnDropper(columns_to_drop)),
    ('numeric_data_imputer', numeric_data_imputer),
    ('categorical_data_imputer', categorical_data_imputer),
])



In [88]:
# BEFORE applying cleaning pipeline
SB_data.isnull().sum()

ESTU_TIPODOCUMENTO               0
ESTU_NACIONALIDAD                0
ESTU_GENERO                      6
ESTU_FECHANACIMIENTO             0
PERIODO                          0
                             ...  
PERCENTIL_ESPECIAL_GLOBAL    77431
ESTU_AGREGADO                64750
ESTU_PRESENTACIONSABADO      64907
SEED_CODIGOMEN               64905
SEED_NOMBRE                  64905
Length: 87, dtype: int64

In [87]:
# AFTER applying cleaning pipeline
cleaned_data = data_cleaning_pipeline.fit_transform(SB_data)
cleaned_data.isnull().sum()


PERIODO                          0
ESTU_COD_RESIDE_DEPTO            0
ESTU_COD_RESIDE_MCPIO            0
COLE_CODIGO_ICFES                0
COLE_COD_DANE_ESTABLECIMIENTO    0
                                ..
ESTU_PRIVADO_LIBERTAD            0
ESTU_MCPIO_PRESENTACION          0
ESTU_DEPTO_PRESENTACION          0
DESEMP_INGLES                    0
ESTU_ESTADOINVESTIGACION         0
Length: 77, dtype: int64

## Conclusiones
- Se creó e implementó un pipeline de limpieza de datos
  - Este pipeline utiliza transformaciones custom para adapatarlo a los requerimientos del proyecto
- Se constrastó la cantidad de valores nulos antes y después del pipeline